<a href="https://colab.research.google.com/github/dotsnangles/Retrieval-Based-Chatbot/blob/main/chatbot_prep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!rm -rf /content/Poly-Encoder

In [3]:
!nvidia-smi

Sun Jul 10 15:35:00 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
# from google.colab import drive
# drive.mount('/content/drive')

In [5]:
!pip install -q transformers datasets folium==0.2.1

In [6]:
!git clone https://github.com/dotsnangles/Poly-Encoder.git
%cd /content/Poly-Encoder

Cloning into 'Poly-Encoder'...
remote: Enumerating objects: 129, done.
remote: Counting objects: 100% (72/72), done.
remote: Compressing objects: 100% (63/63), done.
remote: Total 129 (delta 41), reused 20 (delta 8), pack-reused 57
Receiving objects: 100% (129/129), 40.70 KiB | 10.17 MiB/s, done.
Resolving deltas: 100% (66/66), done.
/content/Poly-Encoder


In [7]:
!gdown -q --folder 1Ipr-aNF5ELMY0HTXAmeV26LlgktKUfmG
!gdown -q --folder 1RH7laK4WlucCw68ZeExFvyg7vs-kB_x3

In [8]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import BertPreTrainedModel, BertConfig, BertModel, BertTokenizer, AutoModel
from encoder import PolyEncoder
from transform import SelectionJoinTransform, SelectionSequentialTransform

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# print(device)

PATH = '/content/Poly-Encoder/chatbot_output/poly_16_pytorch_model.bin'

bert_name = 'klue/bert-base'
bert_config = BertConfig.from_pretrained(bert_name)

tokenizer = BertTokenizer.from_pretrained(bert_name)
tokenizer.add_tokens(['\n'], special_tokens=True)

context_transform = SelectionJoinTransform(tokenizer=tokenizer, max_len=256)
response_transform = SelectionSequentialTransform(tokenizer=tokenizer, max_len=128)

bert = BertModel.from_pretrained(bert_name, config=bert_config)

model = PolyEncoder(bert_config, bert=bert, poly_m=16)
model.resize_token_embeddings(len(tokenizer))
model.load_state_dict(torch.load(PATH))
model.to(device)
model.device

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


device(type='cuda', index=0)

In [9]:
context = ['This framework generates embeddings for each input sentence', 
            'Sentences are passed as a list of string.', 
            'The quick brown fox jumps over the lazy dog.']

candidates = ['This framework generates embeddings for each input sentence', 
            'Sentences are passed as a list of string.', 
            'The quick brown fox jumps over the lazy dog.']

In [10]:
def context_input(context):
    context_input_ids, context_input_masks = context_transform(context)
    contexts_token_ids_list_batch, contexts_input_masks_list_batch = [context_input_ids], [context_input_masks]

    long_tensors = [contexts_token_ids_list_batch, contexts_input_masks_list_batch]

    contexts_token_ids_list_batch, contexts_input_masks_list_batch = (torch.tensor(t, dtype=torch.long, device=device) for t in long_tensors)

    return contexts_token_ids_list_batch, contexts_input_masks_list_batch

In [11]:
contexts_token_ids_list_batch, contexts_input_masks_list_batch = context_input(context)

In [12]:
def response_input(candidates):
    responses_token_ids_list, responses_input_masks_list = response_transform(candidates)
    responses_token_ids_list_batch, responses_input_masks_list_batch = [responses_token_ids_list], [responses_input_masks_list]

    long_tensors = [responses_token_ids_list_batch, responses_input_masks_list_batch]

    responses_token_ids_list_batch, responses_input_masks_list_batch = (torch.tensor(t, dtype=torch.long, device=device) for t in long_tensors)

    return responses_token_ids_list_batch, responses_input_masks_list_batch

In [13]:
responses_token_ids_list_batch, responses_input_masks_list_batch = response_input(candidates)

In [14]:
def embs_gen(contexts_token_ids_list_batch, contexts_input_masks_list_batch):

    with torch.no_grad():
        model.eval()
        
        ctx_out = model.bert(contexts_token_ids_list_batch, contexts_input_masks_list_batch)[0]  # [bs, length, dim]
        poly_code_ids = torch.arange(model.poly_m, dtype=torch.long).to(contexts_token_ids_list_batch.device)
        poly_code_ids = poly_code_ids.unsqueeze(0).expand(1, model.poly_m)
        poly_codes = model.poly_code_embeddings(poly_code_ids) # [bs, poly_m, dim]
        embs = model.dot_attention(poly_codes, ctx_out, ctx_out) # [bs, poly_m, dim]

        return embs

In [15]:
embs = embs_gen(contexts_token_ids_list_batch, contexts_input_masks_list_batch)

In [16]:
def cand_emb_gen(responses_token_ids_list_batch, responses_input_masks_list_batch):

    with torch.no_grad():
        model.eval()
                
        batch_size, res_cnt, seq_length = responses_token_ids_list_batch.shape # res_cnt is 1 during training
        responses_token_ids_list_batch = responses_token_ids_list_batch.view(-1, seq_length)
        responses_input_masks_list_batch = responses_input_masks_list_batch.view(-1, seq_length)
        cand_emb = model.bert(responses_token_ids_list_batch, responses_input_masks_list_batch)[0][:,0,:] # [bs, dim]
        cand_emb = cand_emb.view(batch_size, res_cnt, -1) # [bs, res_cnt, dim]

        return cand_emb

In [17]:
cand_emb = cand_emb_gen(responses_token_ids_list_batch, responses_input_masks_list_batch)

In [19]:
def loss(embs, cand_emb, contexts_token_ids_list_batch, responses_token_ids_list_batch):
    batch_size, res_cnt, seq_length = responses_token_ids_list_batch.shape

    ctx_emb = model.dot_attention(cand_emb, embs, embs) # [bs, bs, dim]
    # print(ctx_emb)
    ctx_emb = ctx_emb.squeeze()
    # print(ctx_emb)
    dot_product = (ctx_emb*cand_emb) # [bs, bs]
    # print(dot_product)
    dot_product = dot_product.sum(-1)
    print(dot_product)
    mask = torch.eye(batch_size).to(contexts_token_ids_list_batch.device) # [bs, bs]
    print(mask)
    loss = F.log_softmax(dot_product, dim=-1)
    print(loss)
    loss = loss * mask
    print(loss)
    loss = (-loss.sum(dim=1))
    print(loss)
    loss = loss.mean()
    print(loss)
    return loss

In [20]:
loss_ = loss(embs, cand_emb, contexts_token_ids_list_batch, responses_token_ids_list_batch)

tensor([[498.3091, 500.8978, 504.1481]], device='cuda:0')
tensor([[1.]], device='cuda:0')
tensor([[-5.8798, -3.2911, -0.0408]], device='cuda:0')
tensor([[-5.8798, -3.2911, -0.0408]], device='cuda:0')
tensor([9.2118], device='cuda:0')
tensor(9.2118, device='cuda:0')


tensor(9.2118, device='cuda:0')

In [22]:
def score(embs, cand_emb):
    with torch.no_grad():
        model.eval()

        ctx_emb = model.dot_attention(cand_emb, embs, embs) # [bs, res_cnt, dim]
        dot_product = (ctx_emb*cand_emb).sum(-1)
        
        return dot_product

In [23]:
score_ = score(embs, cand_emb)

tensor([[498.3091, 500.8978, 504.1481]], device='cuda:0')

In [24]:
# forward
with torch.no_grad():
    model.eval()
    
    model_foward = model(contexts_token_ids_list_batch, contexts_input_masks_list_batch, responses_token_ids_list_batch, responses_input_masks_list_batch)

tensor([[498.3091, 500.8978, 504.1481]], device='cuda:0')


### 데이터 검증

In [25]:
import pickle

with open('/content/Poly-Encoder/감성대화챗봇데이터/train_data_source.pickle', 'rb') as f:
    train = pickle.load(f)
with open('/content/Poly-Encoder/감성대화챗봇데이터/val_data_source.pickle', 'rb') as f:
    dev = pickle.load(f)

In [26]:
# index = 500

In [27]:
# train[index]['context']

In [28]:
# train[index]['responses']

In [29]:
# dev[index]['context']

In [30]:
# dev[index]['responses']

### 챗봇 테이블 생성

In [31]:
data = {
    'context' : [],
    'response': []
}

In [32]:
for sample in train:
    data['context'].append(sample['context'])
    data['response'].append([sample['responses'][0]])

In [33]:
# len(data['context']), len(data['response'])

(40879, 40879)

In [34]:
# idx = 400

In [35]:
# print(data['context'][idx])
# print(data['response'][idx])

['participant 1: 성적이 오르지 않아 고민이었는데 담임 선생님께서 도움을 많이 주셔서 매우 감사해.', 'participant 2: 담임 선생님께서 도움을 많이 주셨군요. 감사한 마음을 어떻게 표현하면 좋을까요?', 'participant 1: 글쎄. 어떻게 전해야 할지 몰라서 정말 고민이야.']
['감사를 표할 방법에 대해 고민되시나 보네요.']


In [36]:
import pandas as pd

In [37]:
df = pd.DataFrame(data)

### generate cand_embs & create tensor table

In [38]:
# response_input_srs = df['response'].apply(response_input)
# response_input_lst = response_input_srs.to_list()

# cand_embs_lst = []
# for sample in response_input_lst:
#     cand_embs_lst.append(cand_emb_gen(*sample).to('cpu'))

In [39]:
# df['response embedding'] = cand_embs_lst
# df[['response', 'response embedding']]

In [40]:
# cand_embs = cand_embs_lst[0]
# for idx in range(1, len(cand_embs_lst)):
#     y = cand_embs_lst[idx]
#     cand_embs = torch.cat((cand_embs, y), 1)

In [41]:
# cand_embs = cand_embs.to(device)

In [42]:
import pickle
with open('/content/Poly-Encoder/감성대화챗봇데이터/cand_embs.pickle', 'rb') as f:
    cand_embs = pickle.load(f)
cand_embs.to(device)

tensor([[[ 0.4922, -0.6527,  0.3834,  ..., -0.2355, -0.0816,  0.7831],
         [ 0.4265, -0.7164,  0.4097,  ..., -0.4494,  0.0708,  1.1537],
         [ 0.3295, -0.6451,  0.3668,  ..., -0.4229,  0.1024,  1.1591],
         ...,
         [ 0.7648, -0.7776,  0.2120,  ..., -0.2492,  0.2255,  0.7545],
         [ 0.0553, -0.9839,  0.3826,  ..., -0.7569, -0.0208,  0.4652],
         [ 0.4029, -0.7795,  0.4000,  ..., -0.3931,  0.0195,  0.8489]]],
       device='cuda:0')

### generate context_embs

In [43]:
query = ['너무 성급한 결정을 한 것 같아.']

In [44]:
embs = embs_gen(*context_input(query))

### Score & Retrieve

In [45]:
import time

In [46]:
start = time.time()
s = score(embs, cand_embs)
end = time.time()

In [47]:
idx = s.argmax(1)

In [48]:
idx = int(idx[0])

In [49]:
df['response'][idx]

['섣부른 결정에 후회하고 계시는군요. 어떻게 하고 싶으세요?']

In [50]:
df.iloc[idx]['context']

['participant 1: 가정 형편에 맞는 차를 계약하려 했는데 자동차 판매사원 판매 전략에 넘어가 버렸어.',
 'participant 2: 예산을 초과하는 금액을 지출하게 되셨군요.',
 'participant 1: 소나타 차량을 생각하고 갔는데 신형 그랜저에 앉아 보고서 마음이 변했지 뭐야.',
 'participant 2: 좋은 걸 사고 싶은 마음과 재정 걱정에 혼란스러우셨겠어요.',
 'participant 1: 집에 와서 다시 생각해보니 잘못 결정한 것 같아.']

### Chatbot UI

In [ ]:
consult_context = {
    'num':[],
    'name': [],
    'customer':[],
    'chatbot':[]
}

print('안녕하세요. 공감 만땅이~~⭐️ 공감이🍀 입니다.')
print('세상에 완벽한 사람 없고, 완벽하지 않은 게 잘못이 아닌 것처럼❌')
print('공감이도 부족한 면이 있지만 당신의 얘기에 집중할꺼에요~!😎')
print('공감이가 당신을 이해할 수 있도록 당신에 대해 길게 말해주세요. (비밀인데 TMI 좋아해요💕)')
print('공감이는 언제나 당신 편입니다. 🥰')

print()
print('-'*50)
print()

while True:

  print('공감이 : ')
  print('종료를 원한다면 "종료"를 입력해주세요.')
  name = str(input('성함이 어떻게 되시나요?: '))
  
  if name == '종료':
    break

  while True:
    print()
    print('공감이🍀 : ')
    confirm = input(f'{name}님 맞으신가요? (네/아니요): ')
    print()

    if confirm == '네':
      break

    else:
      print('공감이🍀 : ')
      name = str(input('성함을 다시 입력해주세요!: '))

  print('-'*50)
  print()

  print(f'{name}님 만약 상담을 그만두고 싶으시다면 "끝"를 입력해주세요.')

  print()
  print(f'{name}님의 고민은 무엇인가요?')
  print()

  count = 1
  
  list_answer = []
  best_num = -1
  
  while True:

    print(f'{name} : ')
    query = [str(input())]
    print()

    best_num = -1
    embs = embs_gen(*context_input(query))
    s = score(embs, cand_embs)
    idx = int(s[0].sort()[-1][best_num])

    best_answer = df['response'][idx][0]

    while True:

      if best_answer not in list_answer:
        break
      
      else:
        best_num += -1
        idx = int(s[0].sort()[-1][best_num])
        best_answer = df['response'][idx][0]

    if query == ['끝']:

      print('-'*50)
      print('-'*50)
      print()

      print(f'공감이🍀는 {name}님이 언제나 행복하시길 바랍니다. 감사합니다')
      print()
      print('-'*50)
      print()
      break

    consult_context['customer'].append(*query)

    print('공감이🍀 : ')
    print(best_answer)
    print()

    consult_context['chatbot'].append(best_answer)
    list_answer.append(best_answer)

    if len(list_answer) == 3:
      del list_answer[0:1]
    
    consult_context['num'].append(count)

    consult_context['name'].append(name)

    count += 1

context = pd.DataFrame(consult_context)